<a href="https://colab.research.google.com/github/Felipanjos/a3_ia_2022.2/blob/main/Projeto_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### D. Preparação dos dados

In [1]:
import pandas as pd
import numpy as np
import random
import string
import sklearn
from itertools import groupby

In [66]:
df = pd.read_csv('2022-2023 NBA Player Stats - Regular.csv', sep=';',encoding='latin')

In [67]:
df = df[['Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA']]

In [68]:
df

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA
0,Precious Achiuwa,C,23,TOR,7,0,22.4,2.9,7.7
1,Steven Adams,C,29,MEM,7,7,25.9,2.3,4.1
2,Bam Adebayo,C,25,MIA,8,8,35.4,6.8,12.6
3,Ochai Agbaji,SG,22,UTA,4,0,14.8,2.0,4.8
4,Santi Aldama,PF,22,MEM,7,7,29.4,3.6,8.3
...,...,...,...,...,...,...,...,...,...
434,Delon Wright,PG,30,WAS,4,0,22.8,2.5,6.0
435,McKinley Wright IV,PG,24,DAL,1,0,5.0,1.0,4.0
436,Thaddeus Young,PF,34,TOR,4,0,8.0,0.8,2.0
437,Trae Young,PG,24,ATL,7,7,35.0,8.9,22.4


In [61]:
np.unique(df['Pos'])

array(['C', 'PF', 'PG', 'SF', 'SG'], dtype=object)

##### a) Selecionando 10% das colunas

No dataset utilizado existem somente 9 colunas, dessa forma:

*    10% de 9 = 0.9
*    Aproximando para 1, fica somente uma coluna a ser selecionada como referência para a modificação dos dados



In [69]:
qtd_colunas = len(df.columns)
p_colunas = round(qtd_colunas * 10 / 100)
coluna_aleatoria = df.sample(n=p_colunas, axis='columns').keys()[0]
colunas_numericas = df.describe().columns
colunas_categoricas = df.describe(exclude=np.number).columns
coluna_aleatoria # escolhendo uma coluna aleatoriamente

'Player'

##### a) Selecionando 3% dos dados

In [71]:
p_3 = int(df.shape[0] * 3 / 100)
p_3 # 3% de 1000 é igual a 30

13

##### a) Excluindo 3% dos dados aleatoriamente

In [72]:
reg_nulos = df[df[coluna_aleatoria].isnull()]
qtd_reg_nulos = reg_nulos.shape[0]
reg_excluidos = df.sample(n=p_3)
reg_exc_index = reg_excluidos.index.array
df.loc[reg_exc_index, coluna_aleatoria] = np.nan

##### a) Alterando 3% dos dados aleatoriamente

In [73]:
def gerar_string_aleatoria():
  letters = string.ascii_lowercase
  return ''.join(random.choice(letters) for i in range(10)) 

def gerar_numero_aleatorio():
  return random.randint(0, 999)

In [74]:
df_selecao_nao_nulo = df[~df[coluna_aleatoria].isnull()]
reg_alterados = df_selecao_nao_nulo.sample(n=p_3)
reg_alter_index = reg_alterados.index.array

In [75]:
if coluna_aleatoria in colunas_numericas:
    for index in reg_alter_index:
        df.loc[[index],[coluna_aleatoria]] = gerar_numero_aleatorio()
else:
    for index in reg_alter_index:
        df.loc[[index],[coluna_aleatoria]] = gerar_string_aleatoria()

#### b) Limpeza dos dados

i. Codificação One-Hot

ii. Dados numéricos ausentes - Atribuição da mediana geral

A atribuição da mediana geral foi utilizada para lidar com os valores nulos das colunas em que existiam, pelos seguintes motivos:

*   Número relativamente pequeno de linhas (1000), o que ocasionaria numa maior escassez de dados caso as linhas com valores nulos fossem removidas
*   Número pequeno de colunas, logo a remoção completa de cada coluna em que houvesse valor nulo acarretaria na perda desnecessária de informações
*   Facilidade na implementação em comparação com o treinamento de predição e atribuição
*   Coesão com o dataset, visto que logo no primeiro quartil a coluna 'Meta_score' apresenta valor próximo à média

Após implementação da correção, o dataset não conta mais com valores nulos.


In [31]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="mean")),        
        ('std_scaler', StandardScaler()),
    ])

In [34]:
from sklearn.preprocessing import OneHotEncoder

num_attribs = list(df.describe().columns)
cat_attribs = ["Genre"]

In [35]:
num_attribs

['IMDB_Rating', 'Meta_score', 'No_of_Votes', 'Gross']

In [36]:
from sklearn.compose import ColumnTransformer

full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", OneHotEncoder(), cat_attribs),
    ])

In [37]:
df_prepared = full_pipeline.fit_transform(df)